In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import gbd_mapping as gbd
from db_queries import get_outputs, get_ids
from get_draws.api import get_draws
from vivarium import Artifact

import bep_summarizer

!whoami
!date

ndbs
Wed Feb  5 13:30:47 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

## Artifact and output directories

In [3]:
artifact_dir = '/ihme/costeffectiveness/artifacts/vivarium_gates_bep/'
results_dir = '/share/costeffectiveness/results/vivarium_gates_bep/base_model_half_day'

locations_rundates = {
    'Mali': '2020_02_04_20_04_06',
    'Tanzania': '2020_02_04_20_09_12'
}

locations = list(locations_rundates.keys())

## GBD IDs from R shiny tool

https://shiny.ihme.washington.edu/content/88/

(Can also use `get_ids`)

In [4]:
# GBD round IDs
gbd_2017 = 5

# Location IDs
tanzania = 189 # United Republic of Tanzania 
mali = 211 # Mali

# Cause IDs
all_causes = 294
diarrhea = 302 # Diarrheal diseases
lri = 322 # Lower respiratory infections
meningitis = 332
measles = 341
neonatal = 380 # Neonatal disorders
pem = 387 # Protein energy malnutrition

# Measures
deaths = 1
prevalence = 5
incidence = 6
emr = 9 # Excess mortality rate
acmr = 14 # All-cause mortality rate
csmr = 15 # Cause-specific mortality rate
relative_risk = 11
birth_prev = 38 # Birth prevalence

# Metrics
number = 1
percent = 2
rate = 3

# Age group IDs
at_birth = 164
early_nn = 2
late_nn = 3
post_nn = 4
one_to_four = 5

# Sex IDs
male=1
female=2
both=3

In [11]:
art_mali = Artifact(artifact_dir + 'mali.hdf', filter_terms=['year_start == 2017', 'age_start < 5'])

## Function to read artifact by draw, since LBWSG data was messed up

In [8]:
def read_lbwsg_data_by_draw(artifact_path, draw):
    key = 'risk_factor/low_birth_weight_and_short_gestation/exposure'
    with pd.HDFStore(artifact_path, mode='r') as store:
        index = store.get(f'{key}/index')
        draw = store.get(f'{key}/draw_{draw}')
    draw = draw.rename("value")
    data = pd.concat([index, draw], axis=1)
    return data

In [35]:
mali_lbwsg = read_lbwsg_data_by_draw(artifact_dir + 'mali.hdf', 0)
mali_lbwsg.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,value
0,Mali,Female,0.0,0.019178,1990,1991,cat10,0.000000
1,Mali,Female,0.0,0.019178,1990,1991,cat106,0.001379
2,Mali,Female,0.0,0.019178,1990,1991,cat11,0.001316
3,Mali,Female,0.0,0.019178,1990,1991,cat116,0.002852
4,Mali,Female,0.0,0.019178,1990,1991,cat117,0.012758


In [36]:
mali_lbwsg.shape

(73416, 8)

In [37]:
mali_lbwsg.age_start.unique()

array([0.000000e+00, 1.917808e-02, 7.671233e-02, 1.000000e+00,
       5.000000e+00, 1.000000e+01, 1.500000e+01, 2.000000e+01,
       2.500000e+01, 3.000000e+01, 3.500000e+01, 4.000000e+01,
       4.500000e+01, 5.000000e+01, 5.500000e+01, 6.000000e+01,
       6.500000e+01, 7.000000e+01, 7.500000e+01, 8.000000e+01,
       8.500000e+01, 9.000000e+01, 9.500000e+01])

## Write function to filter data and read all draws for LBWSG

In [38]:
# Convert a list of filter terms into a pandas .query() string
filter_terms = ['year_start == 2017', 'age_start < 5', "parameter == 'cat10'"]
query_string = " and ".join(filter_terms)
query_string

"year_start == 2017 and age_start < 5 and parameter == 'cat10'"

In [39]:
mali_lbwsg_reduced = mali_lbwsg.query(query_string)
mali_lbwsg_reduced

,location,sex,age_start,age_end,year_start,year_end,parameter,value
1539,Mali,Female,0.000000,0.019178,2017,2018,cat10,0.002128
3135,Mali,Female,0.019178,0.076712,2017,2018,cat10,0.001435
4731,Mali,Female,0.076712,1.000000,2017,2018,cat10,0.000000
6327,Mali,Female,1.000000,5.000000,2017,2018,cat10,0.000000
38247,Mali,Male,0.000000,0.019178,2017,2018,cat10,0.000000
39843,Mali,Male,0.019178,0.076712,2017,2018,cat10,0.000000
41439,Mali,Male,0.076712,1.000000,2017,2018,cat10,0.000000
43035,Mali,Male,1.000000,5.000000,2017,2018,cat10,0.000000


In [44]:
index_cols = [col for col in mali_lbwsg_reduced.columns if col != 'value']
mali_lbwsg_reduced.set_index(index_cols).rename(columns={'value': 'draw_0'})

draw_0
location sex    age_start age_end  year_start year_end parameter          
Mali     Female 0.000000  0.019178 2017       2018     cat10      0.002128
                0.019178  0.076712 2017       2018     cat10      0.001435
                0.076712  1.000000 2017       2018     cat10      0.000000
                1.000000  5.000000 2017       2018     cat10      0.000000
         Male   0.000000  0.019178 2017       2018     cat10      0.000000
                0.019178  0.076712 2017       2018     cat10      0.000000
                0.076712  1.000000 2017       2018     cat10      0.000000
                1.000000  5.000000 2017       2018     cat10      0.000000

In [61]:
def read_lbwsg_data(artifact_path, filter_terms):
    query_string = " and ".join(filter_terms)
    
#     data = read_lbwsg_data_by_draw(artifact_path, 0)
#     data = data.query(query_string)
#     index_cols = [col for col in data.columns if col != 'value']
#     data = data.set_index(index_cols).rename(columns={'value': 'draw_0'})
    draw_data_dfs = []
    
    for draw in range(1000):
        draw_data = read_lbwsg_data_by_draw(artifact_path, draw)
        draw_data = draw_data.query(query_string)
        index_cols = [col for col in data.columns if col != 'value']
        draw_data = draw_data.set_index(index_cols).rename(columns={'value': f'draw_{draw}'})
#         data = pd.concat([data, draw_data], axis=1)
        draw_data_dfs.append(draw_data)
        
#     return data
    return pd.concat(draw_data_dfs, axis=1, copy=False)

In [62]:
mali_lbwsg = read_lbwsg_data(artifact_dir + 'mali.hdf', ['year_start == 2017', 'age_start < 5'])
mali_lbwsg.head()

draw_0  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002128   
                                                       cat106     0.001562   
                                                       cat11      0.001454   
                                                       cat116     0.003774   
                                                       cat117     0.014307   

                                                                    draw_1  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001768   
                                                       cat106     0.001507   
                                                       cat11      0.001369   
                                                       cat116     0.003472   
                                                       cat117     0.014481   

                                                                    draw_2  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001924   
                                                       cat106     0.001917   
                                                       cat11      0.001351   
                                                       cat116     0.003759   
                                                       cat117     0.013703   

                                                                    draw_3  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002649   
                                                       cat106     0.001878   
                                                       cat11      0.001396   
                                                       cat116     0.004055   
                                                       cat117     0.013855   

                                                                    draw_4  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001953   
                                                       cat106     0.002094   
                                                       cat11      0.001431   
                                                       cat116     0.004935   
                                                       cat117     0.016841   

                                                                    draw_5  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002106   
                                                       cat106     0.002062   
                                                       cat11      0.001580   
                                                       cat116     0.004220   
                                                       cat117     0.014736   

                                                                    draw_6  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002140   
                                                       cat106     0.001822   
                                                       cat11      0.001661   
                                                       cat116     0.004160   
                                                       cat117     0.015148   

                                                                    draw_7  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001769   
     

In [55]:
mali_lbwsg.shape

(456, 1000)

### Woo hoo! Looks like the function works

Let's figure out what's in the index:

456 rows = (57 categories) x (4 age groups) x (2 sexes)

In [57]:
# Count the total number of LBWSG categories: 57
mali_lbwsg.index.get_level_values('parameter').nunique()

57

In [58]:
456/57 # 8 = (4 age groups) x (2 sexes)

8.0

## Compute prevalence of LBW

Here are the LBW categories that Ali computed:

In [66]:
# LBW categories found by Ali
lbw_cats = ['cat2','cat810','cat11','cat14','cat15','cat17','cat19','cat20',
            'cat21','cat22','cat23','cat24','cat25','cat26','cat27','cat28',
            'cat29','cat30','cat31','cat32','cat33','cat34','cat35','cat36',
            'cat106','cat116','cat117','cat123']

In [67]:
# Get an index slice for easy indexing of multi-index
idx = pd.IndexSlice

In [69]:
mali_lbw = mali_lbwsg.loc[idx[:,:,:,:,:,:,lbw_cats],:]
mali_lbw.head()

draw_0  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001562   
                                                       cat11      0.001454   
                                                       cat116     0.003774   
                                                       cat117     0.014307   
                                                       cat123     0.004240   

                                                                    draw_1  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001507   
                                                       cat11      0.001369   
                                                       cat116     0.003472   
                                                       cat117     0.014481   
                                                       cat123     0.004098   

                                                                    draw_2  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001917   
                                                       cat11      0.001351   
                                                       cat116     0.003759   
                                                       cat117     0.013703   
                                                       cat123     0.004374   

                                                                    draw_3  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001878   
                                                       cat11      0.001396   
                                                       cat116     0.004055   
                                                       cat117     0.013855   
                                                       cat123     0.004372   

                                                                    draw_4  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.002094   
                                                       cat11      0.001431   
                                                       cat116     0.004935   
                                                       cat117     0.016841   
                                                       cat123     0.005581   

                                                                    draw_5  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.002062   
                                                       cat11      0.001580   
                                                       cat116     0.004220   
                                                       cat117     0.014736   
                                                       cat123     0.004906   

                                                                    draw_6  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001822   
                                                       cat11      0.001661   
                                                       cat116     0.004160   
                                                       cat117     0.015148   
                                                       cat123     0.004864   

                                                                    draw_7  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat106     0.001480   
     

In [70]:
mali_lbw.shape

(216, 1000)

In [71]:
# Compute fraction of LBW categories to total:
216/456

0.47368421052631576

In [74]:
mali_lbw = mali_lbw.groupby(level = ['location', 'sex', 'age_start', 'age_end', 'year_start', 'year_end']).sum()
mali_lbw

draw_0    draw_1  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      0.169228  0.166454   
                0.019178  0.076712 2017       2018      0.166463  0.163515   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   
         Male   0.000000  0.019178 2017       2018      0.181352  0.187546   
                0.019178  0.076712 2017       2018      0.177362  0.183740   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   

                                                          draw_2    draw_3  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      0.173603  0.172370   
                0.019178  0.076712 2017       2018      0.170748  0.169573   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   
         Male   0.000000  0.019178 2017       2018      0.172123  0.188749   
                0.019178  0.076712 2017       2018      0.168536  0.184704   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   

                                                          draw_4    draw_5  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      0.186957  0.176100   
                0.019178  0.076712 2017       2018      0.184043  0.172403   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   
         Male   0.000000  0.019178 2017       2018      0.190954  0.175752   
                0.019178  0.076712 2017       2018      0.186580  0.172025   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   

                                                          draw_6    draw_7  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      0.179044  0.173068   
                0.019178  0.076712 2017       2018      0.176217  0.169966   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   
         Male   0.000000  0.019178 2017       2018      0.183815  0.173795   
                0.019178  0.076712 2017       2018      0.179512  0.170317   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   

                                                          draw_8    draw_9  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.000000  0.019178 2017       2018      0.163172  0.172567   
                0.019178  0.076712 2017       2018      0.160618  0.169745   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   
         Male   0.000000  0.019178 2017       2018      0.165238  0.167023   
                0.019178  0.076712 2017       2018      0.161486  0.163554   
                0.076712  1.000000 2017       2018      0.000000  0.000000   
                1.000000  5.000000 2017       2018      0.000000  0.000000   

                                                        ...  draw_990  \
location sex    age_start age_end  year_start year_end  ...             
Mali   

In [76]:
mali_lbw.T.describe(percentiles=[.025, .5, .975])

location           Mali                                              \
sex              Female                                        Male   
age_start      0.000000     0.019178 0.076712 1.000000     0.000000   
age_end        0.019178     0.076712 1.000000 5.000000     0.019178   
year_start         2017         2017     2017     2017         2017   
year_end           2018         2018     2018     2018         2018   
count       1000.000000  1000.000000   1000.0   1000.0  1000.000000   
mean           0.166297     0.163101      0.0      0.0     0.184576   
std            0.015365     0.015339      0.0      0.0     0.012550   
min            0.110263     0.107150      0.0      0.0     0.133845   
2.5%           0.115074     0.112050      0.0      0.0     0.156051   
50%            0.169477     0.166449      0.0      0.0     0.186376   
97.5%          0.185446     0.181785      0.0      0.0     0.204493   
max            0.188337     0.184491      0.0      0.0     0.208061   

location                                   
sex                                        
age_start      0.019178 0.076712 1.000000  
age_end        0.076712 1.000000 5.000000  
year_start         2017     2017     2017  
year_end           2018     2018     2018  
count       1000.000000   1000.0   1000.0  
mean           0.180686      0.0      0.0  
std            0.012621      0.0      0.0  
min            0.129689      0.0      0.0  
2.5%           0.152069      0.0      0.0  
50%            0.182573      0.0      0.0  
97.5%          0.200375      0.0      0.0  
max            0.203891      0.0      0.0

### Filter down to prevalence in early neonatal to compare with sim output

In [93]:
mali_lbw.index.names

FrozenList(['location', 'sex', 'age_start', 'age_end', 'year_start', 'year_end'])

In [94]:
mali_lbw.loc[idx['Mali', :, 0, :, 2017, :],:]

draw_0    draw_1  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.169228  0.166454   
         Male   0.0       0.019178 2017       2018      0.181352  0.187546   

                                                          draw_2    draw_3  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.173603  0.172370   
         Male   0.0       0.019178 2017       2018      0.172123  0.188749   

                                                          draw_4    draw_5  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.186957  0.176100   
         Male   0.0       0.019178 2017       2018      0.190954  0.175752   

                                                          draw_6    draw_7  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.179044  0.173068   
         Male   0.0       0.019178 2017       2018      0.183815  0.173795   

                                                          draw_8    draw_9  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.163172  0.172567   
         Male   0.0       0.019178 2017       2018      0.165238  0.167023   

                                                        ...  draw_990  \
location sex    age_start age_end  year_start year_end  ...             
Mali     Female 0.0       0.019178 2017       2018      ...  0.132068   
         Male   0.0       0.019178 2017       2018      ...  0.175710   

                                                        draw_991  draw_992  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.167574  0.164557   
         Male   0.0       0.019178 2017       2018      0.191784  0.182988   

                                                        draw_993  draw_994  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.173499  0.172389   
         Male   0.0       0.019178 2017       2018      0.189459  0.178604   

                                                        draw_995  draw_996  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.169476  0.157915   
         Male   0.0       0.019178 2017       2018      0.170764  0.173765   

                                                        draw_997  draw_998  \
location sex    age_start age_end  year_start year_end                       
Mali     Female 0.0       0.019178 2017       2018      0.166064  0.165238   
         Male   0.0       0.019178 2017       2018      0.182690  0.201404   

                                                        draw_999  
location sex    age_start age_end  year_start year_end            
Mali     Female 0.0       0.019178 2017       2018      0.137541  
         Male   0.0       0.019178 2017       2018      0.194077  

[2 rows x 1000 columns]

In [103]:
mali_lbw.index.get_level_values('sex')

Index(['Female', 'Male'], dtype='object', name='sex')

In [104]:
mali_lbw = mali_lbw.loc[idx['Mali', :, 0, :, 2017, :],:]
mali_lbw.index = mali_lbw.index.get_level_values('sex')
mali_lbw

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
sex,,,,,,,,,,,,,,,,,,,,,
Female,0.169228,0.166454,0.173603,0.172370,0.186957,0.176100,0.179044,0.173068,0.163172,0.172567,...,0.132068,0.167574,0.164557,0.173499,0.172389,0.169476,0.157915,0.166064,0.165238,0.137541
Male,0.181352,0.187546,0.172123,0.188749,0.190954,0.175752,0.183815,0.173795,0.165238,0.167023,...,0.175710,0.191784,0.182988,0.189459,0.178604,0.170764,0.173765,0.182690,0.201404,0.194077


In [105]:
mali_lbw.T.describe(percentiles=[.025, .5, .975])

sex,Female,Male
count,1000.000000,1000.000000
mean,0.166297,0.184576
std,0.015365,0.012550
min,0.110263,0.133845
2.5%,0.115074,0.156051
50%,0.169477,0.186376
97.5%,0.185446,0.204493
max,0.188337,0.208061


## Load output to compare LBW prevalence

In [77]:
# Looks like Tanzania is missing two rows (98 vs. 100)
all_output = bep_summarizer.load_by_location_and_rundate(results_dir, locations_rundates)
bep_summarizer.print_location_output_shapes(locations_rundates, all_output)

Mali (100, 1384)
Tanzania (98, 1384)


In [78]:
output = bep_summarizer.BEPOutputSummarizer(all_output)
output.print_column_report()

Number of data columns in output: 1384
Total number of columns captured in categories: 1362

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 1, 'intervention': 0, 'run_time': 0, 'diseases_at_end': 5, 'disease_event_count': 311, 'population': 5, 'person_time': 420, 'treated_days': 0, 'mortality': 210, 'total_daly': 2, 'yld': 180, 'yll': 210, 'categorical_risk': 16, 'graded_sequela': 0} 

Missing (22 data column(s) not captured in a category):
 {'stunting_z_score_sd_at_six_months_among_male', 'gestational_age_proportion_below_37w_among_male', 'gestational_age_mean_among_female', 'stunting_z_score_mean_at_six_months_among_female', 'birth_weight_sd_among_male', 'birth_weight_mean_among_male', 'stunting_z_score_mean_at_six_months_among_male', 'gestational_age_mean_among_male', 'gestational_age_sd_among_male', 'wasting_z_score_mean_at_six_months_among_male', 'neonatal_disorders_prevalent_count_at_birth_among_male', 'stunting_z_score_sd_at_six_months_among

### Select columns with LBWSG data (12 columns)

In [86]:
mali_output = output.data.query("location == 'Mali'")
mali_output.shape

(100, 1384)

In [87]:
mali_output.filter(regex='birth_weight|gestation').columns

Index(['birth_weight_mean_among_male', 'birth_weight_sd_among_male',
       'birth_weight_proportion_below_2500g_among_male',
       'gestational_age_mean_among_male', 'gestational_age_sd_among_male',
       'gestational_age_proportion_below_37w_among_male',
       'birth_weight_mean_among_female', 'birth_weight_sd_among_female',
       'birth_weight_proportion_below_2500g_among_female',
       'gestational_age_mean_among_female', 'gestational_age_sd_among_female',
       'gestational_age_proportion_below_37w_among_female'],
      dtype='object')

In [90]:
mali_lbwsg_output = mali_output.filter(regex='birth_weight|gestation')
mali_lbwsg_output.head()

,,birth_weight_mean_among_male,birth_weight_sd_among_male,birth_weight_proportion_below_2500g_among_male,gestational_age_mean_among_male,gestational_age_sd_among_male,gestational_age_proportion_below_37w_among_male,birth_weight_mean_among_female,birth_weight_sd_among_female,birth_weight_proportion_below_2500g_among_female,gestational_age_mean_among_female,gestational_age_sd_among_female,gestational_age_proportion_below_37w_among_female
input_draw_number,random_seed,,,,,,,,,,,,
650.0,29.0,2902.334353,654.542832,0.184385,38.595780,2.540201,0.141939,2912.806116,680.724978,0.170436,38.499575,2.995673,0.142887
680.0,29.0,2899.760601,655.194657,0.182547,38.598613,2.487803,0.142047,2922.629716,674.754746,0.178769,38.511584,2.744164,0.148701
29.0,5.0,2898.734852,679.047144,0.187404,38.590250,2.555568,0.136905,2914.805046,666.342431,0.176127,38.449692,3.089234,0.156511
602.0,49.0,2927.044303,664.804846,0.172666,38.527370,2.598246,0.153224,2918.784161,671.114632,0.180229,38.470177,3.025045,0.153174
829.0,7.0,2909.163398,665.467295,0.180688,38.623363,2.549753,0.133670,2927.382402,672.171184,0.169792,38.529608,2.841412,0.149598


In [91]:
mali_lbwsg_output.shape

(100, 12)

## Compare LBW proportions in Mali for M/F between sim output and Artifact

### They're close! Woo hoo!

In [106]:
mali_lbwsg_output.filter(regex='birth_weight_proportion').describe(percentiles=[.025, .5, .975])

,birth_weight_proportion_below_2500g_among_male,birth_weight_proportion_below_2500g_among_female
count,100.000000,100.000000
mean,0.181070,0.166331
std,0.009434,0.011472
min,0.161004,0.134976
2.5%,0.163037,0.136603
50%,0.181455,0.168649
97.5%,0.200271,0.184365
max,0.208390,0.186104


In [108]:
# Switch column order to make it easier to compare
mali_lbw.T.describe(percentiles=[.025, .5, .975])[['Male','Female']]

sex,Male,Female
count,1000.000000,1000.000000
mean,0.184576,0.166297
std,0.012550,0.015365
min,0.133845,0.110263
2.5%,0.156051,0.115074
50%,0.186376,0.169477
97.5%,0.204493,0.185446
max,0.208061,0.188337
